In [21]:
# Dependencies
import matplotlib.pyplot as plt
import requests
import json
from pprint import pprint
import pandas as pd
import numpy as np
import time
from datetime import datetime
import xmltodict as xtd
import xml.etree.cElementTree as ET
import zillow
import importlib

# API Keys
from config import gkey, zkey, zws_id

In [2]:
# target_city = input("Please input the city where you would like to move : ")
# radius=int(input("Please input the radius around the city : "))

target_city = 'atlanta'

params = {
    "address": target_city,
    "key": gkey
}

# Build URL using the Google Maps API
base_url = "https://maps.googleapis.com/maps/api/geocode/json"

# Run request
res = requests.get(base_url, params=params).json()

# Convert to JSON
# target_geo = response.json()
# pprint(target_geo)

# Extract lat/lng
lat = res["results"][0]["geometry"]["location"]["lat"]
lng = res["results"][0]["geometry"]["location"]["lng"]
state = res["results"][0]["address_components"][2]["short_name"]

params = {
    "latlng":f"{lat},{lng}",
    "result_type":"postal_code",
    "key":gkey
}

pcode_query = requests.get(f"https://maps.googleapis.com/maps/api/geocode/json", params=params).json()
zip_code = pcode_query['results'][0]['address_components'][0]['long_name']
# Print results
print(f"city:   {target_city}\nlatlng: {lat}, {lng}\nstate:  {state}\nzip:    {zip_code}")

city:   atlanta
latlng: 33.7489954, -84.3879824
state:  GA
zip:    30334


In [3]:
%run ./write_zips.ipynb
import zipcodes
importlib.reload(zipcodes)

<module 'zipcodes' from '/Users/andy/Projects/GTDATA/projects/watson_p1/zipcodes.py'>

In [4]:
zurl = "http://www.zillow.com/webservice/GetRegionChildren.htm"

In [5]:
nparams = {    
    "state":state,
    "childtype":"neighborhood",
    "zws-id": zws_id
}

nres = requests.get(zurl, nparams)


In [16]:
zparams = {
    "state":state,
    "childtype":"zipcode",
    "zws-id":zws_id
}

zres = requests.get(zurl, params=zparams)

In [7]:
ndoc = xtd.parse(nres.text)
zdoc = xtd.parse(zres.text)

In [14]:
ndoc

OrderedDict([('RegionChildren:regionchildren',
              OrderedDict([('@xsi:schemaLocation',
                            'http://www.zillow.com/static/xsd/RegionChildren.xsd https://www.zillowstatic.com/vstatic/18f7df5/static/xsd/RegionChildren.xsd'),
                           ('@xmlns:xsi',
                            'http://www.w3.org/2001/XMLSchema-instance'),
                           ('@xmlns:RegionChildren',
                            'http://www.zillow.com/static/xsd/RegionChildren.xsd'),
                           ('request',
                            OrderedDict([('state', 'Georgia'),
                                         ('childtype', 'neighborhood')])),
                           ('message',
                            OrderedDict([('text',
                                          'Request successfully processed'),
                                         ('code', '0')])),
                           ('response',
                            OrderedDict([('regio

In [15]:
zdoc

OrderedDict([('RegionChildren:regionchildren',
              OrderedDict([('@xsi:schemaLocation',
                            'http://www.zillow.com/static/xsd/RegionChildren.xsd https://www.zillowstatic.com/vstatic/18f7df5/static/xsd/RegionChildren.xsd'),
                           ('@xmlns:xsi',
                            'http://www.w3.org/2001/XMLSchema-instance'),
                           ('@xmlns:RegionChildren',
                            'http://www.zillow.com/static/xsd/RegionChildren.xsd'),
                           ('request',
                            OrderedDict([('state', 'Georgia'),
                                         ('childtype', 'zipcode')])),
                           ('message',
                            OrderedDict([('text',
                                          'Request successfully processed'),
                                         ('code', '0')])),
                           ('response',
                            OrderedDict([('region',
 

In [9]:
ndf = pd.DataFrame(ndoc['RegionChildren:regionchildren']['response']['list']['region'])
ndf.head()

,id,name,url,latitude,longitude,zindex
0,54081,Redan,http://www.zillow.com/local-info/GA-Lithonia/R...,33.738012,-84.156389,NaN
1,269381,Midtown,http://www.zillow.com/local-info/GA-Atlanta/Mi...,33.786845,-84.379061,"{'@currency': 'USD', '#text': '216900'}"
2,269306,Downtown,http://www.zillow.com/local-info/GA-Atlanta/Do...,33.758149,-84.391189,"{'@currency': 'USD', '#text': '149100'}"
3,272907,Belair,http://www.zillow.com/local-info/GA-Augusta/Be...,33.450788,-82.147436,"{'@currency': 'USD', '#text': '118300'}"
4,131530,Saint Simons,http://www.zillow.com/local-info/GA-Saint-Simo...,31.177066,-81.382873,NaN


In [103]:
zdf = pd.DataFrame(zdoc['RegionChildren:regionchildren']['response']['list']['region']).reset_index()
zdf.head()

,index,id,name,zindex,latitude,longitude
0,0,71067,30349,"{'@currency': 'USD', '#text': '83700'}",33.621892,-84.522843
1,1,70829,30044,"{'@currency': 'USD', '#text': '134200'}",33.920832,-84.066282
2,2,70828,30043,"{'@currency': 'USD', '#text': '175800'}",34.008018,-84.0108
3,3,70874,30096,"{'@currency': 'USD', '#text': '163700'}",33.979381,-84.150032
4,4,70810,30024,"{'@currency': 'USD', '#text': '276000'}",34.06545,-84.093791


In [112]:
# for i, p in enumerate(zdf):
#     print(zdf['zindex'][p]['#text'])
# zindex = zcp_df['zindex']
# zcp_df[0]['#text']
# zindex[1]

zips_prices_df = zdf[['name', 'zindex']]
prices = zips_prices_df['zindex']
prices
# for i,p in enumerate(zips_prices_df):
#     print(prices[i]['#text'])
#     print(zips_prices_df[name][i], zips_prices_df['zindex'][i]['#text'])

0       {'@currency': 'USD', '#text': '83700'}
1      {'@currency': 'USD', '#text': '134200'}
2      {'@currency': 'USD', '#text': '175800'}
3      {'@currency': 'USD', '#text': '163700'}
4      {'@currency': 'USD', '#text': '276000'}
5      {'@currency': 'USD', '#text': '119800'}
6      {'@currency': 'USD', '#text': '235200'}
7      {'@currency': 'USD', '#text': '172700'}
8      {'@currency': 'USD', '#text': '111400'}
9      {'@currency': 'USD', '#text': '183100'}
10     {'@currency': 'USD', '#text': '157700'}
11     {'@currency': 'USD', '#text': '289900'}
12     {'@currency': 'USD', '#text': '254900'}
13     {'@currency': 'USD', '#text': '105000'}
14     {'@currency': 'USD', '#text': '338100'}
15     {'@currency': 'USD', '#text': '340500'}
16     {'@currency': 'USD', '#text': '143800'}
17     {'@currency': 'USD', '#text': '126800'}
18     {'@currency': 'USD', '#text': '138500'}
19     {'@currency': 'USD', '#text': '110400'}
20     {'@currency': 'USD', '#text': '137800'}
21      {'@cu

In [ ]:
api = zillow.ValuationApi()
zpid="2100641621"
detail_data = api.GetZEstimate(zws_id, zpid)
address = detail_data.full_address.get_dict()
address['zipcode']
# adf = pd.DataFrame(address, index=[0])
# adf

In [ ]:
zdf = detail_data.get_dict()
pd.DataFrame(zdf).head()

In [ ]:
address = "3400 Pacific Ave., Marina Del Rey, CA"
postal_code = "90292"

data = api.GetDeepSearchResults(zws_id, address, postal_code)

In [ ]:
dsr = data.get_dict()
pd.DataFrame(dsr).head()